In [1]:
from pyMuellerMat import common_mms as cmm
from pyMuellerMat import MuellerMat
import numpy as np
import matplotlib.pyplot as plt

#Some nicer printing options to avoid machine errors
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

#### First we'll generate a wollaston prism MuellerMatrix object and a half-wave plate MuellerMatrix object

In [2]:
wollaston_mm = cmm.WollastonPrism()
hwp_mm = cmm.HWP()

#### Now we'll combine them into a SystemMuellerMatrix

In [3]:
sys_mm = MuellerMat.SystemMuellerMatrix([wollaston_mm,hwp_mm])

#### Let's take a look at the possible keywords to the System

In [6]:
sys_mm.master_property_dict

{'HalfwaveRetarder': {'theta': 0.0},
 'WollastonPrism': {'beam': 'o', 'eta': 1.0, 'theta': 0.0}}

The HWP only has one parameter - the rotation angle theta.  
The WollastonPrism has two paramters - The 'beam' can either be 'o' for ordinary or 'e' for extraordinary

#### We'll evaluate the system mueller matrix with the default parameters above

In [7]:
m1 = sys_mm.evaluate()
print(m1)

[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]


#### What does it look like if we want to look at the extraordinary beam

In [8]:
sys_mm.master_property_dict['WollastonPrism']['beam']='e'

In [9]:
m2 = sys_mm.evaluate()
print(m2)

[[ 0.5 -0.5  0.   0. ]
 [-0.5  0.5  0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]]


#### Cool! Normally we'll take the difference of the ordinary and extraordinary beam. 

In [10]:
m_sys1_diff = m1-m2
m_sys1_sum = m1+m2
print("Different Mueller Matrix")
print(m_sys1_diff)
print("\n Sum Mueller Matrix")
print(m_sys1_sum)

Different Mueller Matrix
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

 Sum Mueller Matrix
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


#### Super! That's what it should look like. Now we can rotate the HWP by 45 degrees to swap the beam:

In [12]:
sys_mm.master_property_dict['HalfwaveRetarder']['theta']=45
sys_mm.master_property_dict['WollastonPrism']['beam']='o'
m1 = sys_mm.evaluate()
sys_mm.master_property_dict['WollastonPrism']['beam']='e'
m2 = sys_mm.evaluate()

In [13]:
m_sys2_diff = m1-m2
m_sys2_sum = m1+m2
print("Different Mueller Matrix")
print(m_sys2_diff)
print("\n Sum Mueller Matrix")
print(m_sys2_sum)

Different Mueller Matrix
[[ 0. -1.  0. -0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]

 Sum Mueller Matrix
[[ 1.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


Great! The sign has flipped on the Q component, as we wanted. 

#### Let's generate some fake sum and difference data with a random input Stokes Vector
Since we've only generated WP angles that are sensitive to Q, we'll leave U blank

In [14]:
## Choose an input and make some fake measurements
S=[1,-0.5,0,0]

ipsum = np.matmul(m_sys1_sum,S)[0]
ipdiff = np.matmul(m_sys1_diff,S)[0]
insum = np.matmul(m_sys2_sum,S)[0]
indiff = np.matmul(m_sys2_diff,S)[0]

measurements = [ipsum, ipdiff,insum,indiff]
print(measurements)

[1.0, -0.5, 1.0, 0.5]


The data is displayed as [sum0,diff0,sum45,diff45]

#### Generate a measurement matrix and take the inverse! 

In [15]:
measurement_matrix = np.vstack([m_sys1_sum[0],m_sys1_diff[0],
                               m_sys2_sum[0],m_sys2_diff[0]])
print(measurement_matrix)

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0. -1.  0. -0.]]


In [16]:
inv_measurement_matrix = np.linalg.pinv(measurement_matrix)
print(inv_measurement_matrix)

[[ 0.5  0.   0.5  0. ]
 [ 0.   0.5  0.  -0.5]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]]


#### Finally, recover our input polarization.

In [17]:
S_in = np.matmul(inv_measurement_matrix,measurements)
print(S_in)

[ 1.  -0.5  0.   0. ]
